In [1]:
!pip install torch_geometric


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [3]:
# Imports
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
# PyG
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split

# Vostri moduli
from src.loadData    import (GraphDataset, TestDataset)
from src.models      import GNN
from src.transforms import EdgeDropout, NodeDropout, Compose
from src.losses import (
    GeneralizedCELoss,
    SymmetricCELoss,
    estimate_transition_matrix,
    ForwardCorrectionLoss,
    BootstrappingLoss
)
from src.divide_mix_def import DivideMixTrainer

# Fissa seed
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)



In [4]:
def add_zeros(data):
    if not hasattr(data, 'x') or data.x is None:
        data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [5]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    
    model = model.to(device)
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    print(f"Train loss/acc: {total_loss / len(data_loader):.4f}/{correct / total:.4f}")
    return total_loss / len(data_loader),  correct / total

In [6]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)

            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())

    
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
        print(f"Test loss/acc {total_loss / len(data_loader):.4f} / {correct / total:.4f}")
    return predictions

In [7]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd()
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))

    os.makedirs(submission_folder, exist_ok=True)

    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")

    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })

    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [8]:
def plot_training_progress(train_losses, train_accuracies, save_plot, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    if(save_plot):
        # Save plots in the current directory
        os.makedirs(output_dir, exist_ok=True)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "training_progress.png"))
        plt.close()

In [9]:
# Hyper-parameters
device                =  "cuda" if torch.cuda.is_available() else "cpu"
# Modello
gnn_type              = 'gin-virtual'   
num_layer             = 3
emb_dim               = 300
drop_ratio            = 0.7

edge_p  = 0.2   # frazione di bordi da droppare
node_p = 0.2

lr                    = 0.0005
epochs                = 50
weight_decay          = 5e-4
t_max                 = 50
eta_min               = 1e-5
num_classes           = 6
batch_size            = 32 
patience              = 12
warmup_epochs         = 10
lambda_u              = 1
T                     = 0.6
alpha                 = 6
p_threshold           = 0.8

transforms = Compose([
    EdgeDropout(p=edge_p),
    NodeDropout(p=node_p),
    add_zeros,
])

# Epoch 49: Train Loss=0.3052, Train Acc=0.9117, Val Acc=0.6953 DATASET C

# Divide-Mix

In [10]:
def create_gnn_model(gnn_type, num_classes, num_layer, emb_dim, drop_ratio, device, residual):
    kwargs = {
        'gnn_type': gnn_type.replace("-virtual", ""),
        'num_class': num_classes,
        'num_layer': num_layer,
        'emb_dim': emb_dim,
        'drop_ratio': drop_ratio,
        'virtual_node': "virtual" in gnn_type,
        'residual': residual
        }

    model = GNN(**kwargs).to(device)
    return model


def create_optimizer_and_scheduler(model, lr=lr, weight_decay=weight_decay, t_max=t_max, eta_min=eta_min):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=t_max, eta_min=eta_min)
    return optimizer, scheduler



In [11]:
def split_dataset(dataset: GraphDataset, val_ratio=0.1, seed=42):
    labels = torch.tensor([data.y.item() for data in dataset])
    indices = list(range(len(dataset)))

    train_idx, val_idx = train_test_split(
        indices,
        test_size=val_ratio,
        stratify=labels,
        random_state=seed
    )

    train_subset = []
    val_subset = []

    for new_idx, original_idx in enumerate(train_idx):
        data = dataset[original_idx]
        data.idx = new_idx  # Normalize index
        train_subset.append(data)

    for new_idx, original_idx in enumerate(val_idx):
        data = dataset[original_idx]
        data.idx = new_idx  # Normalize index
        val_subset.append(data)

    return train_subset, val_subset


In [12]:
import gc 
# Modifica loop principale di training
train_datasets_path = ["datasets/B/train.json.gz"]

for ds in train_datasets_path:


    print("Generating dataset")
    
    full_dataset = GraphDataset(ds, transform=transforms).shuffle()
    #full_dataset = TestDataset()
    print("Splitting dataset")
    train_set, val_set = split_dataset(full_dataset, 0.2, 42)
    

    model_kwargs = {"gnn_type": "gin-virtual", "num_classes":num_classes, "num_layer": num_layer, "emb_dim": emb_dim, "drop_ratio": drop_ratio, "device":device, "residual": False }
    optim_kwargs = {"lr": lr, "weight_decay": weight_decay, "t_max": t_max, "eta_min": eta_min}
    criterion = nn.CrossEntropyLoss(reduction="none")
    trainer = DivideMixTrainer(
        model_creator=create_gnn_model,
        model_kwargs=model_kwargs, 
        criterion=criterion,
        optim_sched_creator=create_optimizer_and_scheduler,
        optim_sched_kwargs=optim_kwargs,
        device=device,
        num_models=2,
        warmup_epochs=warmup_epochs,
        lambda_u=lambda_u, 
        T = T,
        alpha = alpha,
        p_threshold=p_threshold,
    )

    # Train
    best_model = trainer.train_full_pipeline(
        train_dataset=train_set,
        val_dataset=val_set,
        epochs=epochs,
        batch_size=batch_size,
        patience = patience
    )

    del val_set, train_set, trainer, full_dataset
    gc.collect()

    # Test e predizioni
    test_loader = DataLoader(GraphDataset(ds.replace("train", "test"), transform=transforms), batch_size=32)
    #test_loader = DataLoader(TestDataset(), batch_size=batch_size)
    predictions = DivideMixTrainer.predict_ensemble(best_model, test_loader, device)
    save_predictions(predictions=predictions, test_path=ds.replace("train", "test"))
    del test_loader, criterion, best_model, predictions
    gc.collect()
    torch.cuda.empty_cache()

Generating dataset
Splitting dataset
Warmup phase (10 Epochs)


100%|██████████| 140/140 [00:04<00:00, 29.43it/s]


End of Warmup Stage


100%|██████████| 140/140 [00:09<00:00, 14.61it/s]


Epoch 1/50 | Sup=1.6524 | Unsup=1.3841 | TrainAcc=0.3636 | ValAcc=0.3875 (Best=0.0000)


...

Epoch 29/50 | Sup=1.3948 | Unsup=0.6610 | TrainAcc=0.4609 | ValAcc=0.4348 (Best=0.4366)


100%|██████████| 140/140 [00:09<00:00, 14.11it/s]


Epoch 30/50 | Sup=1.3987 | Unsup=0.6494 | TrainAcc=0.4518 | ValAcc=0.4321 (Best=0.4366)
Early stopping triggered!
Evaluating model 0


 49%|████▉     | 24/49 [00:02<00:02, 11.90it/s]


KeyboardInterrupt: 